# JUMPMAN 23 

### Importing Libraries

In [37]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from folium import plugins
import folium


In [38]:
import plotly.offline as py 
import plotly.graph_objs as go
import cufflinks as cf
import plotly.figure_factory as ff
import plotly.express as px
cf.go_offline()
py.init_notebook_mode(connected=True)

In [39]:
# Reading the input file

df = pd.read_csv("analyze_me.csv")

### Initial review of data

In [40]:
df.head()

,delivery_id,customer_id,jumpman_id,vehicle_type,pickup_place,place_category,item_name,item_quantity,item_category_name,how_long_it_took_to_order,pickup_lat,pickup_lon,dropoff_lat,dropoff_lon,when_the_delivery_started,when_the_Jumpman_arrived_at_pickup,when_the_Jumpman_left_pickup,when_the_Jumpman_arrived_at_dropoff
0,1457973,327168,162381,van,Melt Shop,American,Lemonade,1.0,Beverages,00:19:58.582052,40.744607,-73.990742,40.752073,-73.985370,2014-10-26 13:51:59.898924,NaN,NaN,2014-10-26 14:52:06.313088
1,1377056,64452,104533,bicycle,Prince Street Pizza,Pizza,Neapolitan Rice Balls,3.0,Munchables,00:25:09.107093,40.723080,-73.994615,40.719722,-73.991858,2014-10-16 21:58:58.65491,2014-10-16 22:26:02.120931,2014-10-16 22:48:23.091253,2014-10-16 22:59:22.948873
2,1476547,83095,132725,bicycle,Bareburger,Burger,Bare Sodas,1.0,Drinks,00:06:44.541717,40.728478,-73.998392,40.728606,-73.995143,2014-10-28 21:39:52.654394,2014-10-28 21:37:18.793405,2014-10-28 21:59:09.98481,2014-10-28 22:04:40.634962
3,1485494,271149,157175,bicycle,Juice Press,Juice Bar,OMG! My Favorite Juice!,1.0,Cold Pressed Juices,NaN,40.738868,-74.002747,40.751257,-74.005634,2014-10-30 10:54:11.531894,2014-10-30 11:04:17.759577,2014-10-30 11:16:37.895816,2014-10-30 11:32:38.090061
4,1327707,122609,118095,bicycle,Blue Ribbon Sushi,Japanese,Spicy Tuna & Tempura Flakes,2.0,Maki (Special Rolls),00:03:45.035418,40.726110,-74.002492,40.709323,-74.015867,2014-10-10 00:07:18.450505,2014-10-10 00:14:42.702223,2014-10-10 00:25:19.400294,2014-10-10 00:48:27.150595


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5983 entries, 0 to 5982
Data columns (total 18 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   delivery_id                          5983 non-null   int64  
 1   customer_id                          5983 non-null   int64  
 2   jumpman_id                           5983 non-null   int64  
 3   vehicle_type                         5983 non-null   object 
 4   pickup_place                         5983 non-null   object 
 5   place_category                       5100 non-null   object 
 6   item_name                            4753 non-null   object 
 7   item_quantity                        4753 non-null   float64
 8   item_category_name                   4753 non-null   object 
 9   how_long_it_took_to_order            3038 non-null   object 
 10  pickup_lat                           5983 non-null   float64
 11  pickup_lon                    

## DATA INTEGRITY

In [42]:
# Finding and handling missing values

df.isnull().sum()


delivery_id                               0
customer_id                               0
jumpman_id                                0
vehicle_type                              0
pickup_place                              0
place_category                          883
item_name                              1230
item_quantity                          1230
item_category_name                     1230
how_long_it_took_to_order              2945
pickup_lat                                0
pickup_lon                                0
dropoff_lat                               0
dropoff_lon                               0
when_the_delivery_started                 0
when_the_Jumpman_arrived_at_pickup      550
when_the_Jumpman_left_pickup            550
when_the_Jumpman_arrived_at_dropoff       0
dtype: int64

#### Missing value imputation

In [43]:
# For categorical variables most common values are imputed for missing values

df.place_category=df.place_category.fillna('Italian')
df.place_category.value_counts()

df.item_category_name=df.item_category_name.fillna('Sides')
df.item_category_name.value_counts()

Sides                                    1423
Burgers                                   148
Appetizers                                145
Sandwiches                                123
Fries                                     111
                                         ... 
Munchables                                  1
Ensaladas                                   1
Salads (Priced by 1/4, 1/2, whole lb)       1
Digestion & Nausea                          1
Market Sides                                1
Name: item_category_name, Length: 767, dtype: int64

In [44]:
df.describe()


,delivery_id,customer_id,jumpman_id,item_quantity,pickup_lat,pickup_lon,dropoff_lat,dropoff_lon
count,5.983000e+03,5983.000000,5983.000000,4753.000000,5983.000000,5983.000000,5983.000000,5983.000000
mean,1.379495e+06,176472.595521,102661.602541,1.248264,40.741425,-73.987101,40.744216,-73.985765
std,6.459397e+04,116414.487842,48607.211795,0.785904,0.022833,0.014896,0.025251,0.018060
min,1.271706e+06,242.000000,3296.000000,1.000000,40.665611,-74.015837,40.649356,-74.017679
25%,1.322792e+06,77817.000000,60761.000000,1.000000,40.724340,-73.996630,40.725307,-74.000297
50%,1.375689e+06,131093.000000,113364.000000,1.000000,40.735677,-73.988682,40.740424,-73.989280
75%,1.436371e+06,293381.000000,143807.000000,1.000000,40.758726,-73.980739,40.763885,-73.974696
max,1.491424e+06,405547.000000,181543.000000,16.000000,40.818082,-73.920980,40.848324,-73.924124


In [45]:
# TO find duplicate rows

df["is_duplicate"]= df.duplicated()
df.is_duplicate.value_counts(dropna=False)

False    5967
True       16
Name: is_duplicate, dtype: int64

In [46]:
df=df.drop_duplicates()


There are totally 16 duplicate records identified and deleted

#### Outlier analysis

In [47]:
df['how_long_it_took_to_order'] = pd.to_datetime(df.how_long_it_took_to_order, format = '%H:%M:%S.%f')

def mins(val):
    return val.minute*60.0 + val.second
minute = df.how_long_it_took_to_order.apply(mins)
df['min_to_order'] = round(minute/60,2)


In [48]:

fig = px.violin(df, y="min_to_order")
fig.show()

There are multiple orders which took more than 30 mins to place an order . These orders can be considered outliers and needs to be investigated to see if there is any performance issues with the app.   

The other data integrity issue that needs attention is the columns when_the_Jumpman_arrived_at_pickup and when_the_Jumpman_left_pickup have exact same number of null values indicating some of the orders doesnt record the datetime attrribute of the delivery and must be inspected further.



#### Changing from Object  to DateTime format 

In [49]:
df.when_the_delivery_started= pd.to_datetime(df.when_the_delivery_started)
df.when_the_Jumpman_arrived_at_pickup = pd.to_datetime(df.when_the_Jumpman_arrived_at_pickup)
df.when_the_Jumpman_left_pickup = pd.to_datetime(df.when_the_Jumpman_left_pickup)
df.when_the_Jumpman_arrived_at_dropoff= pd.to_datetime(df.when_the_Jumpman_arrived_at_dropoff)

#### Calculating multiple durations during a delivery 


In [50]:

df['pickup_arrival_duration'] = df.when_the_Jumpman_arrived_at_pickup - df.when_the_delivery_started
df['pickup_waiting_duration'] = df.when_the_Jumpman_left_pickup - df.when_the_Jumpman_arrived_at_pickup
df['dropoff_duration'] = df.when_the_Jumpman_arrived_at_dropoff-df.when_the_Jumpman_left_pickup


In [51]:
df.how_long_it_took_to_order = pd.to_datetime(df.how_long_it_took_to_order, format = '%H:%M:%S.%f')

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5983 entries, 0 to 5982
Data columns (total 23 columns):
 #   Column                               Non-Null Count  Dtype          
---  ------                               --------------  -----          
 0   delivery_id                          5983 non-null   int64          
 1   customer_id                          5983 non-null   int64          
 2   jumpman_id                           5983 non-null   int64          
 3   vehicle_type                         5983 non-null   object         
 4   pickup_place                         5983 non-null   object         
 5   place_category                       5983 non-null   object         
 6   item_name                            4753 non-null   object         
 7   item_quantity                        4753 non-null   float64        
 8   item_category_name                   5983 non-null   object         
 9   how_long_it_took_to_order            3038 non-null   datetime64[ns] 
 10  

### Understanding Popularity through Visualization

#### Popular pickup place with most orders


In [63]:

pop_places = df.groupby('pickup_place').sum()
pop_places.sort_values('item_quantity', ascending = 0).item_quantity[:10].iplot(kind='bar',title='Most Popular Pickup places',
                                                                                colors='Blue')


#### Most Ordered items


In [53]:

pop_items = df.groupby('item_name').sum()
pop_items.sort_values('item_quantity', ascending = 0).item_quantity[:10].iplot(kind='bar',title='Most Ordered Items'
                                                                               ,colors='green')

#### Most Popular hours 

In [54]:
def gethrs(row):
    return row.hour

df.when_the_delivery_started.apply(gethrs).value_counts().iplot(kind='bar', title='Most popular hours',color='red')


#### Pickup place having most waiting time 


In [55]:
def days_hours_minutes(td):
    return (td.seconds//60)%60

df['waiting_min']=df.pickup_waiting_duration.apply(days_hours_minutes)


In [56]:
pop_places = df.groupby('pickup_place').sum()
pop_places.sort_values('waiting_min', ascending = 0).waiting_min[:10].iplot(kind='bar',title='Pickup place with Most waiting time',
                                                                                colors='Blue')

These 3 popularity metrics are essential for our analysis. It would  improve efficiency when we pre-order in following cases
1. With more than 6000 minutes of overall pickup waiting time as well as being the most popular pickup place, Preordering from Shake Shack would increase 
2. Also other most ordered foods like Fries during the busy hours will help us to reduce the waiting time for JumpMen and also alleviates order congestion.
From the graph, we can infer that the demand is high during Dinner time between 6 pm to 8 pm 

#### Most popular mode of vehicle by JumpMen

In [57]:
df.vehicle_type.value_counts(normalize=0)


bicycle       4274
car           1215
walker         274
van             76
scooter         75
truck           48
motorcycle      21
Name: vehicle_type, dtype: int64

### HEATMAPS

In [58]:
m = folium.Map([40.744607, -73.990742], zoom_start=12)
n = folium.Map([40.744607, -73.990742], zoom_start=12)

#### Pickup heatzone

In [59]:
stationArr = df[['pickup_lat', 'pickup_lon']].to_numpy()

# plot heatmap
m.add_child(plugins.HeatMap(stationArr, radius=15))
m

#### Dropoff Heatzone

In [60]:
stationArr = df[['dropoff_lat', 'dropoff_lon']].to_numpy()

# plot heatmap
n.add_child(plugins.HeatMap(stationArr, radius=15))
n

Dropoff activity is much more spread out than Restaurant Partners activity (goes out into Queens and Brooklyn and upper Manhattan)


#### Location changes each hour 

In [61]:
df['hour'] = df.when_the_delivery_started.apply(gethrs)
hour_15D = df.loc[df.hour ==15] 

In [62]:
hour_15 = folium.Map([40.744607, -73.990742], zoom_start=12)
stationArr = hour_15D[['pickup_lat', 'pickup_lon']].to_numpy()

# plot heatmap
hour_15.add_child(plugins.HeatMap(stationArr, radius=15))
hour_15